In [110]:
# https://github.com/ElinaChhabra/Trilateration-in-3d/blob/master/trilateration.py
# https://stackoverflow.com/questions/1406375/finding-intersection-points-between-3-spheres

In [111]:
import numpy as np                                         
from numpy import sqrt, dot, cross                       
from numpy.linalg import norm                            

# Find the intersection of three spheres                 
# P1,P2,P3 are the centers, r1,r2,r3 are the radii       
# Implementaton based on Wikipedia Trilateration article.                              
def trilaterate(P1,P2,P3,r1,r2,r3):                      
    temp1 = P2-P1                                        
    e_x = temp1/norm(temp1)                              
    temp2 = P3-P1                                        
    i = dot(e_x,temp2)                                   
    temp3 = temp2 - i*e_x                                
    e_y = temp3/norm(temp3)                              
    e_z = cross(e_x,e_y)                                 
    d = norm(P2-P1)                                      
    j = dot(e_y,temp2)                                   
    x = (r1*r1 - r2*r2 + d*d) / (2*d)                    
    y = (r1*r1 - r3*r3 -2*i*x + i*i + j*j) / (2*j)
    temp4 = r1*r1 - x*x - y*y
    if abs(temp4) < 1e-6:
        raise ArithmeticError("Exact intersection found. Algorithm does not work in this case.")
    elif temp4 < 0:                                          
        raise ValueError("The three spheres do not intersect.")
    z = sqrt(temp4)                                      
    p_12_a = P1 + x*e_x + y*e_y + z*e_z                  
    p_12_b = P1 + x*e_x + y*e_y - z*e_z                  
    return p_12_a, p_12_b

In [189]:
points.shape, anchors.shape, line_lengths.shape, 


((1000, 3), (4, 3), (4, 1000))

In [281]:
from scipy.optimize import minimize
from collections import defaultdict

line_init = np.array([100, 200, 300, 100])
# hat_line_lengths = line_lengths - line_init[:, None]

measured_points = np.array([
                            [50., 50., 10.], [60., 50., 10.], [50., 60., 10.], [40., 50., 10.], [50., 40., 10.],
                            [55., 55., 10.], [45., 55., 10.], [45., 45., 10.], [55., 45., 10.],
                            [50., 50., 50.], [70., 50., 50.], [50., 70., 50.], [30., 50., 15.], [50., 30., 15.],
                            ])
################################################################
sim_measured_points = simulate_points(5000, 50, np.array([50, 50, 50]))
grid_points = np.concatenate([measured_points, sim_measured_points], axis=0)
################################################################
measured_line_lengths = find_line_lengths(grid_points, anchors)
# measured_line_lengths -= line_init[:, None]


In [282]:
# distances_to_center(grid_points[0], anchors, measured_line_lengths[:,0])

In [224]:
from scipy.optimize import minimize

def multilat_to_dist_err(point, anchors, line_lengths):
    return ((np.linalg.norm(point - anchors, axis=1) - np.abs(line_lengths))**2).sum()

# def distances_to_center(point, anchors, line_lengths):
#     # point of size 3 dim
#     # line_lengths of number of anchors dim
#     return ((np.linalg.norm(point - anchors, axis=1) - line_lengths)**2).sum()

def multilaterate_error(anchors, line_lengths):
    point = minimize(multilat_to_dist_err,
                     x0=np.ones(3)*50,
                     args=(anchors, line_lengths))
    if point.success:
        return multilat_to_dist_err(point.x, anchors, line_lengths)
    else:
        return None
    
# def multilaterate_error_alg(anchors, line_lengths):
#     a1, a2, a3, a4 = anchors
#     d1, d2, d3, d4 = line_lengths
#     x1 = a1 - np.sqrt(-a2**2 + 2*a2*x2 - a3**2 + 2*a3*x3 - a4**2 + 2 a4 x4 + d**2 - x2**2 - x3**2 - x4**2)

def multilaterate_errors(x, obs_line_lengths, num_points_grid=None, num_points_torque=None):
    anchors = x[:4*3].reshape(4, 3)
    line_init = x[4*3:4*3+4]
    errors = []
    for obs_line_length in obs_line_lengths.T[:num_points_torque]:  # for every data point
        multilat_err = multilaterate_error(anchors, obs_line_length + line_init)
        # skip if failure
        if multilat_err is None:
            continue
        errors.append(multilat_err)
    mean_error = np.array(errors).nanmean()
    return mean_error

def euclidean_estimate(obs_line_lengths, method, options):
    sol = minimize(multilaterate_errors,
                   args=(obs_line_lengths,),
                   method='BFGS',
                   options={'gtol': 1e-3, 'disp': True, 'maxiter': 1e5},
                   tol=1e-3,
                   x0=np.concatenate([anchors.flatten(), line_lengths[:,0].flatten()])
    )
    if sol.success:
        return sol.x
    else:
        return None

In [852]:
np.mean([np.nan, 1, 2, 3], )

nan

In [236]:
assert np.allclose(multilaterate_errors(np.concatenate([anchors.flatten(), line_init.flatten()]), measured_line_lengths), 0)

In [113]:
P1 = np.array([0, 0, 0])
P2 = np.array([6, 0, 0])
P3 = np.array([3, 3, 0])
r1 = 3
r2 = 3
r3 = 3
try:
    trilaterate(P1,P2,P3,r1,r2,r3)  # fails
except ArithmeticError:
    print('Exact intersection.')

Exact intersection.


In [114]:
P1 = np.array([0, 0, 0])
P2 = np.array([5.9999, 0, 0])
P3 = np.array([3, 3, 0])
r1 = 3
r2 = 3
r3 = 3
try:
    p1, p2 = trilaterate(P1,P2,P3,r1,r2,r3)  # fails
except ArithmeticError:
    print('Exact intersection.')
print(p1, p2)
print(np.linalg.norm(p1-p2))

[2.99995000e+00 5.00000000e-05 1.73203637e-02] [ 2.99995000e+00  5.00000000e-05 -1.73203637e-02]
0.034640727474987816


In [115]:
anchors = np.array(
    [
        [0, 0, -10],
        [100, 0, -10],
        [50, 100, -10],
        [50, 50, 100]
        ]
    )

In [116]:
def simulate_points(n_points, radius, center=np.array([50, 50, 50])):
    def sample(n_points, radius):
        oversample_proportion = 1 + (4/3 * np.pi * radius**3) / (2 * radius)**3
        points = np.random.rand(int(n_points * oversample_proportion), 3) * 2 * radius
        points -= [radius, radius, radius]
        accepted_points = points[np.linalg.norm(points, axis=1) < radius]
        return accepted_points
    points = sample(n_points, radius)[:n_points]
    while len(points) < n_points:
        new_points = sample(n_points - len(points), radius)
        points = np.concatenate([points, new_points], axis=0)
    points = points + center
    return points[:n_points]

points = simulate_points(50, radius=10)
points

array([[49.10654192, 50.12237093, 48.06493647],
       [48.76943602, 45.78532115, 43.20561633],
       [46.90599109, 55.26556547, 57.01523976],
       [54.09989681, 54.68264465, 51.36214907],
       [54.11623779, 50.69270532, 43.18071202],
       [45.86980736, 44.64174164, 55.7898519 ],
       [57.71702843, 54.84355323, 51.30438641],
       [54.00162518, 48.67983039, 51.00746696],
       [55.25077438, 43.2781916 , 45.79155213],
       [48.56676502, 57.11314158, 55.50336373],
       [58.76092392, 49.46563002, 49.45217806],
       [47.3464051 , 51.44012346, 59.52181206],
       [45.60773059, 52.00692779, 51.83507629],
       [46.48125223, 55.10045466, 51.69521405],
       [51.62929037, 54.29004762, 48.26518105],
       [58.64543499, 54.05077482, 52.34790076],
       [49.60861766, 44.7327685 , 45.84576178],
       [47.29052566, 44.08822001, 54.97186403],
       [56.73895579, 52.62081491, 46.11472843],
       [54.59955661, 55.77471539, 48.87366506],
       [53.59705586, 42.97413146, 47.436

In [117]:
def find_line_lengths(points, anchors):
    line_lengths = []
    for anchor in anchors:
        line_lengths.append(np.linalg.norm(points - anchor, axis=1))
    line_lengths = np.array(line_lengths)
    return line_lengths
line_lengths = find_line_lengths(points, anchors)
line_lengths

array([[ 91.0782157 ,  85.47275081,  98.71928427,  98.39869816,
         91.24968874,  91.78904584, 100.48531482,  94.90949602,
         89.65683034,  99.5556742 ,  97.1300994 ,  98.59538899,
         92.78126044,  94.88343559,  94.90955809, 101.23257397,
         87.06770143,  91.65974076,  95.58829433,  97.76522335,
         89.54564873,  98.04313592,  89.54432475,  86.67055425,
         99.39851735,  88.07453249,  93.50140587,  83.28538582,
         91.20471817, 100.15480303,  89.61194809,  97.4711638 ,
         92.66673404,  88.24298419,  99.27338703,  96.74799656,
         90.72817392,  82.87391137,  91.75657301,  91.52620289,
         92.98724043,  93.49250165,  96.61162336,  90.31852154,
         83.06188664,  99.52091572,  93.74347996,  85.29606719,
         90.49195755,  87.65738343],
       [ 92.05396782,  86.90054044, 101.8052006 ,  94.13991947,
         86.6213492 ,  96.18350932,  92.48725755,  90.59518419,
         83.59540867, 100.98504474,  87.6474268 , 101.25102326,
   

In [118]:
# add noise to anchors and lengths
noisy_anchors = anchors * (1 + (np.random.rand(*anchors.shape)-0.5)/10)
noisy_line_lengths = line_lengths * (1 + (np.random.rand(*line_lengths.shape)-0.5) * 1/10)

In [119]:
import itertools
def find_intersections(anchors, line_lengths):
    intersections_by_point = []
    for point_index in range(line_lengths.shape[1]):
        intersections_by_anchor_combo = []
        inter_trilat_point_errors = []
        for anchor_index in itertools.combinations(range(len(anchors)), 3):
            # TODO: generalize to case when there's more than 1 validation anchor (ie len(anchors) > 4)
            anchor_subset = anchors[list(anchor_index)]
            anchor_val_index = np.setdiff1d(np.arange(line_lengths.shape[0]), anchor_index)
            anchor_val = anchors[anchor_val_index]
            
            try:
                intersections_i = trilaterate(*anchor_subset, *line_lengths[anchor_index, point_index])
            except ValueError:
                intersections_by_anchor_combo.append([np.nan, np.nan, np.nan])
                inter_trilat_point_errors.append(np.nan)
                continue
                # print('Warning: No intersection possible, attempting with added noise.')
                # intersections_noisy = []
                # intersection_count = 0
                # j = 0
                # while intersection_count < 20 and j < 1000:
                #     try:
                #         # add noise, make noise grow slowly
                #         line_lengths_noisy = line_lengths * (1 + (np.random.rand(*line_lengths.shape)-0.5) * (j+1)**(1/3)/100)
                #         intersections_noisy.append(trilaterate(*anchor_subset, *line_lengths_noisy[anchor_index, i]))
                #         intersection_count += 1
                #         j += 1
                #     except:
                #         j += 1
                # if intersection_count == 0:
                #     raise ValueError('No intersections found.')
                # elif intersection_count < 20:
                #     print('Warning: low count of random intersections. Convergence might be bad.')
                # intersections_i = np.array(intersections_noisy).mean(axis=0)
            # tie-brake between points using the 4th anchor
            dist0 = np.linalg.norm(anchor_val-intersections_i[0])
            dist1 = np.linalg.norm(anchor_val-intersections_i[1])
            radius_val = line_lengths[anchor_val_index, point_index]
            error_between_single_trilat_points = np.abs(radius_val-dist0) - np.abs(radius_val-dist1)
            if error_between_single_trilat_points < 0:
                weighted_intersection = intersections_i[0] * 0.9 + intersections_i[1] * 0.1
            else:
                weighted_intersection = intersections_i[1] * 0.9 + intersections_i[0] * 0.1
            inter_trilat_point_errors.append(abs(error_between_single_trilat_points.item()))
            intersections_by_anchor_combo.append(weighted_intersection)

        nan_filter = ~np.isnan(np.array(intersections_by_anchor_combo)).all(axis=1)
        intersections_by_anchor_combo = np.array(intersections_by_anchor_combo)[nan_filter]
        inter_trilat_point_errors = np.array(inter_trilat_point_errors)[nan_filter]
        point_estimate = np.average(intersections_by_anchor_combo, weights=inter_trilat_point_errors, axis=0)
        intersections_by_point.append(np.array(point_estimate))

    return np.array(intersections_by_point)

estimated_intersections_anchor = find_intersections(noisy_anchors, noisy_line_lengths)

estimated_intersections = estimated_intersections_anchor.mean(axis=0)
estimated_intersections

array([52.40658206, 50.40480604, 39.29233448])

In [120]:
find_line_lengths([[50,50,0]], anchors).T[0][[0, 1, 3]]

array([ 71.41428429,  71.41428429, 100.        ])

In [122]:
trilaterate(np.array([0,0,-10]), [100,0,-10], [50, 50, 100], 71.41428429, 71.41428429, 100)

(array([ 50.        , -25.34246576,  34.24657535]),
 array([ 5.00000000e+01,  5.00000000e+01, -1.42108547e-14]))

In [124]:
P1 = np.array([0, 0, 0])
P2 = np.array([5.9999, 0, 0])
P3 = np.array([3, 3, 0])
r1 = 3
r2 = 3
r3 = 3
try:
    p1, p2 = trilaterate(*anchors[[0,1,2]], *line_lengths.T[0][[0,1,2]].tolist())  # fails
except ArithmeticError:
    print('Exact intersection.')    
print(p1, p2)
print(np.linalg.norm(p1-p2))

[49.10654192 50.12237093 48.06493647] [ 49.10654192  50.12237093 -68.06493647]
116.12987294691305


In [125]:
# import itertools
# def get_errors_between_trilats(anchors, line_lengths):
#     intersections_by_point = []
#     errors = []
#     for point_index in range(line_lengths.shape[1]):
#         intersections_by_anchor_combo = []
#         inter_trilat_point_errors = []
#         for anchor_index in itertools.combinations(range(len(anchors)), 3):
#             # TODO: generalize to case when there's more than 1 validation anchor (ie len(anchors) > 4)
#             anchor_subset = anchors[list(anchor_index)]
#             anchor_val_index = np.setdiff1d(np.arange(line_lengths.shape[0]), anchor_index)
#             anchor_val = anchors[anchor_val_index]
            
#             try:
#                 try:
#                     intersections_i = trilaterate(*anchor_subset, *line_lengths[anchor_index, point_index])
#                 except ValueError:
#                     intersections_by_anchor_combo.append([np.nan, np.nan, np.nan])
#                     inter_trilat_point_errors.append(np.nan)
#                     continue
#             except ArithmeticError:
#                 inter_trilat_point_errors.append(0.)
#                 # print('Warning: No intersection possible, attempting with added noise.')
#                 # intersections_noisy = []
#                 # intersection_count = 0
#                 # j = 0
#                 # while intersection_count < 20 and j < 1000:
#                 #     try:
#                 #         # add noise, make noise grow slowly
#                 #         line_lengths_noisy = line_lengths * (1 + (np.random.rand(*line_lengths.shape)-0.5) * (j+1)**(1/3)/100)
#                 #         intersections_noisy.append(trilaterate(*anchor_subset, *line_lengths_noisy[anchor_index, i]))
#                 #         intersection_count += 1
#                 #         j += 1
#                 #     except:
#                 #         j += 1
#                 # if intersection_count == 0:
#                 #     raise ValueError('No intersections found.')
#                 # elif intersection_count < 20:
#                 #     print('Warning: low count of random intersections. Convergence might be bad.')
#                 # intersections_i = np.array(intersections_noisy).mean(axis=0)
#             # tie-brake between points using the 4th anchor
#             dist0 = np.linalg.norm(anchor_val-intersections_i[0])
#             dist1 = np.linalg.norm(anchor_val-intersections_i[1])
#             radius_val = line_lengths[anchor_val_index, point_index]
#             error_between_single_trilat_points = np.abs(radius_val-dist0) - np.abs(radius_val-dist1)
#             if error_between_single_trilat_points < 0:
#                 weighted_intersection = intersections_i[0] * 0.9 + intersections_i[1] * 0.1
#             else:
#                 weighted_intersection = intersections_i[1] * 0.9 + intersections_i[0] * 0.1
#             inter_trilat_point_errors.append(abs(error_between_single_trilat_points.item()))
#             intersections_by_anchor_combo.append(weighted_intersection)

#         nan_filter = ~np.isnan(np.array(intersections_by_anchor_combo)).all(axis=1)
#         intersections_by_anchor_combo = np.array(intersections_by_anchor_combo)[nan_filter]
#         inter_trilat_point_errors = np.array(inter_trilat_point_errors)[nan_filter]
#         point_estimate = np.average(intersections_by_anchor_combo, weights=inter_trilat_point_errors, axis=0)
#         intersections_by_point.append(np.array(point_estimate))
#         errors.append(inter_trilat_point_errors.sum())

#     return np.array(errors)

# trilat_errors_noisy = get_errors_between_trilats(noisy_anchors, noisy_line_lengths)
# trilat_errors = get_errors_between_trilats(anchors, line_lengths)

# # estimated_intersections = estimated_intersections_anchor.mean(axis=0)
# # estimated_intersections
# print(trilat_errors_noisy)
# print(trilat_errors)


[151.81880145 175.40408403 169.05376243 163.02363717 191.86722696
 159.33600953 153.15823437 148.03563606 156.63454458 179.34830214
 171.2049141  159.72830495 159.83844501 164.09817765 145.21471858
 170.20294085 104.2454243  120.41943    128.40740936 148.73510512
 183.65311444 148.65247191 140.98883629 108.10921555 145.464387
 145.11443343 182.70476023 107.02026887 136.37815391 146.26505609
 168.5333952  152.45527197 150.55013362 129.479474   182.32203391
 177.07191342 177.0346655  127.26574323 180.3108721  127.79415721
 116.61412072 178.07672803 153.28258059 109.04136229  92.67014383
 156.44351131 163.54958261 154.81694946 131.982222    92.87414795]
[192.61964214 191.14483287 195.15554848 193.31520993 191.25899558
 193.7241007  192.59653886 193.02665831 190.92629997 194.69431359
 191.74133633 196.01108021 193.40352667 193.47058088 192.71638774
 192.66047665 191.56753471 193.53211341 191.51801235 192.5264612
 191.41706919 194.98492631 192.08480743 191.63521588 195.09018306
 191.3871626

In [193]:
# sample_sizes = [1, 10, 100, 1000]

# anchors = np.array(
#     [
#         [0, 0, 0],
#         [100, 0, 0],
#         [50, 100, 0],
#         [50, 50, 100]
#         ]
#     )

# points = simulate_points(max(sample_sizes), radius=5, center=[50, 50, 50])
# line_lengths = find_line_lengths(points, anchors)

# # add noise to anchors and lengths
# noisy_anchors = anchors + (np.random.rand(*anchors.shape)-0.5)*10/5 * 10
# noisy_anchors = np.array(
#     [
#         [0, 0, 0],
#         [100, 0, 0],
#         [50, 100, 0],
#         [51, 49, 105]
#         ]
#     )
# # noisy_line_lengths = line_lengths #* (1 + (np.random.rand(*line_lengths.shape)-0.5) * 1/100)

# scores = []
# for num_points in sample_sizes:
#     estimated_intersections_anchor = find_intersections(noisy_anchors, line_lengths[:, :num_points])
#     estimated_intersections = np.nanmean(estimated_intersections_anchor, axis=0)  # average across anchors
#     estimated_intersections_sigma = np.nanstd(estimated_intersections_anchor, axis=0)
#     errors = np.linalg.norm(points[:num_points] - estimated_intersections, axis=1)
#     distances = []
#     for point_combo in itertools.combinations(range(len(estimated_intersections_anchor)), 2):
#         point_diff = estimated_intersections_anchor[point_combo[0]] - estimated_intersections_anchor[point_combo[1]]
#         distance = np.linalg.norm(point_diff, axis=1)
#         distances.append(distance)
#     inter_intersection_distance = np.nanmean(np.array(distances))
#     scores.append((num_points, inter_intersection_distance, errors.mean(), estimated_intersections_sigma.mean()))
# scores

/var/folders/92/xt0q79l947bcltlw2c8gyr1h0000gn/T/ipykernel_1047/3014330465.py:38: RuntimeWarning: Mean of empty slice
  inter_intersection_distance = np.nanmean(np.array(distances))


AxisError: axis 1 is out of bounds for array of dimension 1

In [539]:
def euclidean(x, hat_line_lengths, measured_points, num_points_grid, num_points_torque):
    anchors = x[:4*3].reshape(4, 3)
    line_init = x[4*3:4*3+4]

    line_lengths = find_line_lengths(measured_points[:num_points_grid], anchors[:, :num_points_grid])
    sqerror = (line_lengths - (hat_line_lengths[:, :num_points_grid] + line_init[:, None]))**2
    return sqerror.sum()  # sum across anchors and data points

In [540]:
def euclidean_anchor(x, line_init, hat_line_lengths, measured_points, num_points_grid, num_points_torque):
    x = np.concatenate([x, line_init])
    return euclidean(x, hat_line_lengths, measured_points, num_points_grid, num_points_torque)

def euclidean_anchor_d(x, line_init, hat_line_lengths, measured_points, num_points_grid, num_points_torque):
    x = np.concatenate([x[:-1], line_init[:-1], x[-1:]])
    return euclidean(x, hat_line_lengths, measured_points, num_points_grid, num_points_torque)

In [ ]:
# a = np.zeros_like(anchors)
# a[0,0] = 0.
# ll = find_line_lengths(measured_points, anchors+a)
# x = np.concatenate([anchors.flatten(), np.array([0, 0, 0, 0])], axis=0)

# euclidean(x, hat_line_lengths=ll, measured_points=measured_points)

In [551]:
def inter_anchor_combo_distance(x, hat_line_lengths, num_points=20, measured_points=[]):
    anchors = x[:4*3].reshape(4, 3)
    line_init = x[4*3:4*3+4]

    line_lengths = hat_line_lengths[:, :num_points] + line_init[:, None]
    try:
        estimated_intersections_anchor = find_intersections(anchors, line_lengths)
    except ArithmeticError:
        return 0.
    distances = []
    for point_combo in itertools.combinations(range(len(estimated_intersections_anchor)), 2):
        # point_diff = estimated_intersections_anchor[point_combo[0]] - estimated_intersections_anchor[point_combo[1]]
        # distance_between_trilat_points = np.linalg.norm(point_diff, axis=1)
        distances.append(distance_between_trilat_points)
    distances = np.array(distances)
    mean_distance = np.nanmean(distances)
    if np.isnan(mean_distance):
        return 9999.
    else:
        return mean_distance

In [552]:
def inter_anchor_and_euclidean(x, measured_line_lengths, measured_points, num_points_grid, num_points_torque):
    # print(x)
    inter = multilaterate_errors(x, measured_line_lengths[:, num_points_grid:], num_points_grid=None, num_points_torque=num_points_torque)
    euc = euclidean(x, measured_line_lengths[:, :num_points_grid], measured_points[:num_points_grid], num_points_grid=num_points_grid, num_points_torque=None)
    # print('inter', inter)
    # print('euc', euc)

    return euc + inter

In [553]:
def inter_anchor_and_euclidean_anchor(x, line_init, measured_line_lengths, measured_points, num_points_grid, num_points_torque):
    x = np.concatenate([x, line_init])
    return inter_anchor_and_euclidean(x, measured_line_lengths, measured_points, num_points_grid, num_points_torque)

def inter_anchor_and_euclidean_line_init(x, anchor, measured_line_lengths, measured_points, num_points_grid, num_points_torque):
    x = np.concatenate([anchor, x])
    return inter_anchor_and_euclidean(x, measured_line_lengths, measured_points, num_points_grid, num_points_torque)

In [874]:
from scipy.optimize import minimize
from collections import defaultdict

line_init = np.array([100, 200, 300, 100])
# hat_line_lengths = line_lengths - line_init[:, None]

measured_points = np.array([
                            [50., 50., 10.], [70., 50., 10.], [50., 70., 10.], [30., 50., 10.], [50., 30., 10.],
                            [70., 70., 10.], [30., 70., 10.], [30., 30., 10.], [70., 30., 10.],
                            [50., 50., 50.], [70., 50., 50.], [50., 70., 50.], [30., 50., 15.], [50., 30., 15.],
                            ])
################################################################
sim_measured_points = simulate_points(5000, radius=40, center=np.array([50, 50, 50]))
grid_points = np.concatenate([measured_points, sim_measured_points], axis=0)
################################################################
measured_line_lengths = find_line_lengths(grid_points, anchors)
grid_points[:measured_points.shape[0], :measured_points.shape[1]] += (np.random.rand(*measured_points.shape)-0.5)*2 * 3*4.5e-5 # encoder precision is 4.5e-5
measured_line_lengths -= line_init[:, None]

In [575]:
assert np.allclose(inter_anchor_and_euclidean(np.concatenate([anchors.flatten(), line_init.flatten()]), measured_line_lengths, measured_points, num_points_grid, num_points_torque),0)

In [880]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

# grid search
options = {
        #    'Powell': {'gtol': 1e-10, 'disp': True, 'maxiter': 1e3},
        #    'SLSQP': {'disp': True, 'maxiter': 1e5},
        #    'Nelder-Mead': {'maxfev': 50000, 'return_all': True, 'maxiter': 1e3},
           'BFGS': {'gtol': 1e-5, 'disp': True, 'maxiter': 1e5},
        #    'CG': {'gtol': 1e-2, 'disp': True, 'maxiter': 1e5},
           }

num_points_grid = 5
num_points_torque = 100
from scipy.optimize import basinhopping, differential_evolution

sols = defaultdict(list)
for i in range(1):
    # different anchor locations
    x0 = np.concatenate([(anchors + (np.random.rand(*anchors.shape)-0.5)*2 * 10).flatten(),
                        #  np.array([105, 185, 280, 90])])
                         np.array([100., 201, 300, 150])])
    # x0 = np.concatenate([(anchors).flatten(),
    #                     np.array([100, 200, 100, 90])])
    for method in options.keys():
            # sols[(method, loss)].append(
                # perhaps a two part iteration of cables + anchors would work better
                # minimize(inputs[loss][0], args=(measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                #          method=method,
                #          options=options[method],
                #          tol=1e-5,
                #          x0=x0)
                # for i in range(10):
                    # res = minimize(inter_anchor_and_euclidean_line_init, args=(x0[:4*3], measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                    #         method=method,
                    #         options=options[method],
                    #         tol=1e-5,
                    #         x0=x0[4*3:4*3+4])
                    # x0 = np.concatenate([x0[:4*3], res.x])
                    # if inter_anchor_and_euclidean(x0, measured_line_lengths, grid_points, num_points_grid, num_points_torque) < 1e-4:
                    #      break
                    # res = minimize(euclidean_anchor, args=(x0[4*3:4*3+4], measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                    #         method=method,
                    #         options=options[method],
                    #         tol=1e-5,
                    #         x0=x0[:4*3])
                    # x0 = np.concatenate([res.x, x0[4*3:4*3+4]])
                    res = minimize(euclidean_anchor_d, args=(x0[4*3:4*3+4], measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                            method=method,
                            options=options[method],
                            tol=1e-5,
                            x0=np.concatenate([x0[:4*3], x0[-1:]])
                    )
                    x0 = np.concatenate([res.x[:-1], x0[4*3:4*3+4-1], res.x[-1:]])
                    print(x0[:4*3].reshape(4,3))
                    print(x0[4*3:])
                    # 1: up to ~0.50 cum line error in ABC, ~0.2 in D with 5 grid points
                    if (metric:=inter_anchor_and_euclidean(x0, measured_line_lengths, grid_points, num_points_grid, num_points_torque)) < 1:
                        print('SUCCESS', metric)
                    else:
                        print('IMPRECISE', metric)
                        print('INCREASING NUMBER OF SAMPLES')
                        num_points_grid = 10
                        # def stopper_callback(x, f, accepted):
                        #      print(f, flush=True)
                        #      return f < 1e-5
                        # res = basinhopping(
                        #             inter_anchor_and_euclidean,
                        #             minimizer_kwargs={'method': method,
                        #                               'args': (measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                        #                               'options': {**options[method],
                        #                                           'maxiter': 2, 'gtol': 1e-2},
                        #                               'tol': 1e-1},
                        #             T=10,
                        #             stepsize=6,
                        #             callback=stopper_callback,
                        #             x0=x0)
                        NUM_ITER = 10
                        i = 0
                        x0_prev = x0.copy()
                        while True:
                            res = minimize(euclidean, args=(measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                                method=method,
                                options={**options[method],
                                        'maxiter': 1e4,
                                        'gtol': 1e-4},
                                tol=1e-5,
                                x0=np.concatenate([(x0_prev[:4*3] + (np.random.rand(4*3)-0.5)*2 * 40).flatten(),
                                                    x0_prev[4*3:4*3+4]])
                            )
                            x0 = res.x
                            print(x0[:4*3].reshape(4,3))
                            print(x0[4*3:])
                            if i == NUM_ITER and (metric:=inter_anchor_and_euclidean(x0, measured_line_lengths, grid_points, num_points_grid, num_points_torque)):
                                 print('FAILED', metric)
                                 break
                            if (metric:=inter_anchor_and_euclidean(x0, measured_line_lengths, grid_points, num_points_grid, num_points_torque)) < 1:
                                print('SUCCESS', metric)
                                break
                            else:
                                print('IMPRECISE, RETRYING', metric)
                            i += 1
                # differential_evolution(shorty, x0=x0, bounds=[[0, 100] for _ in range(12)] + [[-500, 500] for _ in range(4)])
            # )

Optimization terminated successfully.
         Current function value: 0.000325
         Iterations: 69
         Function evaluations: 1036
         Gradient evaluations: 74
[[0.000 0.000 -0.001]
 [100.680 -0.681 -0.447]
 [50.000 100.000 -0.000]
 [50.000 50.000 99.999]]
[100.000 201.000 300.000 99.999]
SUCCESS 0.0008318202897691233


In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

# grid search
options = {
        #    'Powell': {'gtol': 1e-10, 'disp': True, 'maxiter': 1e3},
        #    'SLSQP': {'disp': True, 'maxiter': 1e5},
        #    'Nelder-Mead': {'maxfev': 50000, 'return_all': True, 'maxiter': 1e3},
           'BFGS': {'gtol': 1e-5, 'disp': True, 'maxiter': 1e5},
        #    'CG': {'gtol': 1e-2, 'disp': True, 'maxiter': 1e5},
           }
inputs = {
        #   'euclidean': (euclidean,),
        #   'inter': (inter_anchor_combo_distance, line_lengths - line_init[:, None]),
          'both': (inter_anchor_and_euclidean,),
         }

num_points_grid = 5
num_points_torque = 100
from scipy.optimize import basinhopping, differential_evolution

sols = defaultdict(list)
for i in range(1):
    # different anchor locations
    x0 = np.concatenate([(anchors + (np.random.rand(*anchors.shape)-0.5)*2 * 10).flatten(),
                        #  np.array([105, 185, 280, 90])])
                         np.array([100., 201, 300, 150])])
    # x0 = np.concatenate([(anchors).flatten(),
    #                     np.array([100, 200, 100, 90])])
    for method in options.keys():
        for loss in inputs.keys():
            # sols[(method, loss)].append(
                # perhaps a two part iteration of cables + anchors would work better
                # minimize(inputs[loss][0], args=(measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                #          method=method,
                #          options=options[method],
                #          tol=1e-5,
                #          x0=x0)
                # for i in range(10):
                    # res = minimize(inter_anchor_and_euclidean_line_init, args=(x0[:4*3], measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                    #         method=method,
                    #         options=options[method],
                    #         tol=1e-5,
                    #         x0=x0[4*3:4*3+4])
                    # x0 = np.concatenate([x0[:4*3], res.x])
                    # if inter_anchor_and_euclidean(x0, measured_line_lengths, grid_points, num_points_grid, num_points_torque) < 1e-4:
                    #      break
                    # res = minimize(euclidean_anchor, args=(x0[4*3:4*3+4], measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                    #         method=method,
                    #         options=options[method],
                    #         tol=1e-5,
                    #         x0=x0[:4*3])
                    # x0 = np.concatenate([res.x, x0[4*3:4*3+4]])
                    res = minimize(euclidean_anchor_d, args=(x0[4*3:4*3+4], measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                            method=method,
                            options=options[method],
                            tol=1e-5,
                            x0=np.concatenate([x0[:4*3], x0[-1:]])
                    )
                    x0 = np.concatenate([res.x[:-1], x0[4*3:4*3+4-1], res.x[-1:]])
                    print(x0[:4*3].reshape(4,3))
                    print(x0[4*3:])
                    # 1: up to ~0.50 cum line error in ABC, ~0.2 in D with 5 grid points
                    if (metric:=inter_anchor_and_euclidean(x0, measured_line_lengths, grid_points, num_points_grid, num_points_torque)) < 1:
                        print('SUCCESS', metric)
                    else:
                        print('IMPRECISE', metric)
                        print('INCREASING NUMBER OF SAMPLES')
                        num_points_grid = 10
                        # def stopper_callback(x, f, accepted):
                        #      print(f, flush=True)
                        #      return f < 1e-5
                        # res = basinhopping(
                        #             inter_anchor_and_euclidean,
                        #             minimizer_kwargs={'method': method,
                        #                               'args': (measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                        #                               'options': {**options[method],
                        #                                           'maxiter': 2, 'gtol': 1e-2},
                        #                               'tol': 1e-1},
                        #             T=10,
                        #             stepsize=6,
                        #             callback=stopper_callback,
                        #             x0=x0)
                        NUM_ITER = 10
                        i = 0
                        x0_prev = x0.copy()
                        while True:
                            res = minimize(euclidean, args=(measured_line_lengths, grid_points, num_points_grid, num_points_torque),
                                method=method,
                                options={**options[method],
                                        'maxiter': 1e4,
                                        'gtol': 1e-4},
                                tol=1e-5,
                                x0=np.concatenate([(x0_prev[:4*3] + (np.random.rand(4*3)-0.5)*2 * 40).flatten(),
                                                    x0_prev[4*3:4*3+4]])
                            )
                            x0 = res.x
                            print(x0[:4*3].reshape(4,3))
                            print(x0[4*3:])
                            if i == NUM_ITER and (metric:=inter_anchor_and_euclidean(x0, measured_line_lengths, grid_points, num_points_grid, num_points_torque)):
                                 print('FAILED', metric)
                                 break
                            if (metric:=inter_anchor_and_euclidean(x0, measured_line_lengths, grid_points, num_points_grid, num_points_torque)) < 1:
                                print('SUCCESS', metric)
                                break
                            else:
                                print('IMPRECISE, RETRYING', metric)
                            i += 1
                # differential_evolution(shorty, x0=x0, bounds=[[0, 100] for _ in range(12)] + [[-500, 500] for _ in range(4)])
            # )

Optimization terminated successfully.
         Current function value: 0.000325
         Iterations: 71
         Function evaluations: 1064
         Gradient evaluations: 76
[[0.000 0.000 -0.001]
 [100.680 -0.681 -0.447]
 [50.000 100.000 -0.000]
 [50.000 50.000 99.999]]
[100.000 201.000 300.000 99.999]
SUCCESS 0.000831935572752305


In [848]:
# sensitivity analysis

# 1 in init = 5
# 1 in anchors ABC = 2-5 xy, 0.1 y
# 1 in anchor D = 0.1 xy, 5 y

# 0.5 in init = 1.4
# 0.5 in anchors ABC = 0.07-1 xy, 0.04 y
# 0.5 in anchor D = 0.03 xy, 1 y

# 0.1 in init = 0.05
# 0.1 in anchors ABC = 0.003-0.05 xy, 0.002 y
# 0.1 in anchor D = 0.001 xy, 0.05 y

anchors_trial = \
np.array([[  0,   0,   .1],
       [100,   0,   0],
       [ 50, 100,   0],
       [ 50,  50, 100]])

line_init_trial = \
np.array([100, 200, 300, 100])

x0 = np.concatenate([anchors_trial.flatten(), line_init_trial.flatten()])
inter_anchor_and_euclidean(x0, measured_line_lengths, grid_points, num_points_grid, num_points_torque)

0.001620846652679436

In [579]:
anchors

array([[  0,   0,   0],
       [100,   0,   0],
       [ 50, 100,   0],
       [ 50,  50, 100]])

Objective: Determine a good cut-off for inter_anchor_and_euclidean values to pass as valid solution. Evaluate inter_anchor_and_euclidean function by passing exact anchor values and play with line lengths and vice-versa. Also, eval joint effect of changing both values

# above

In [ ]:
num_points_grid = 14
max_retries = 10


options = {
        #    'Powell': {'gtol': 1e-10, 'disp': True, 'maxiter': 1e3},
        #    'SLSQP': {'gtol': 1e-10, 'disp': True, 'maxiter': 1e3},
        #    'Nelder-Mead': {'maxfev': 50000, 'return_all': True, 'maxiter': 1e4},
           'BFGS': {'gtol': 1e-3, 'disp': True, 'maxiter': 1e5},
        #    'CG': {'gtol': 1e-3, 'disp': True, 'maxiter': 1e4},
           }
inputs = {
          'euclidean': [euclidean, measured_line_lengths[:, :num_points_grid]],
        #   'inter': [inter_anchor_combo_distance, line_lengths - line_init[:, None]] ,
        #   'both': [inter_anchor_and_euclidean, np.concatenate([measured_line_lengths, hat_line_lengths], axis=1)],
         }

gp = grid_points[:num_points_grid]

num_failures = 0
retry_failure = 0

loss = 'euclidean'

sols = defaultdict(list)
og_input = find_line_lengths(grid_points, anchors)
for point_index in range(35):
    grid_shaky_points = grid_points + (np.random.rand(*grid_points.shape)-0.5)*2 * 0.2
    shaky_lengths = find_line_lengths(grid_shaky_points, anchors)[:, :num_points_grid] - line_init[:, None]
    inputs[loss][1] = shaky_lengths
    measured_anchors = (anchors + (np.random.rand(*anchors.shape)-0.5)*2 * 20).flatten()
    for num_try in range(max_retries):
        print('Attempt: ', num_try+1)
        # different anchor locations
        x0 = np.concatenate([measured_anchors,
                            #  (line_init + (np.random.rand(*line_init.shape)-0.5)*2 * 200).flatten(),
                            1000 * np.random.rand(*line_init.shape),
                            ])
        for method in options.keys():
            for loss in inputs.keys():
                sol = minimize(inputs[loss][0], args=(inputs[loss][1], num_points, gp),
                                method=method,
                                options=options[method],
                                tol=1e-3,
                                x0=x0)
        if (euc:=euclidean(sol.x, hat_line_lengths=inputs[loss][1][:, :measured_line_lengths.shape[1]], measured_points=gp, num_points=num_points)) > 0.7 \
                and num_try + 1 < max_retries:  # 1e-3 for theoretical
            print('Euclidean loss: ', euc)
            num_failures += 1
            continue
        else:
            if num_try + 1 == max_retries:
                retry_failure += 1
            sols[(method, loss)].append(dict(
                sol=sol,
                anchor_measure_error=np.abs(sol.x[:np.multiply(*anchors.shape)] - anchors.flatten()).mean(),
                line_measure_error=np.abs(sol.x[np.multiply(*anchors.shape):] - line_init).mean(),
                anchor_loss = ((sol.x[:np.multiply(*anchors.shape)].reshape(*anchors.shape) - anchors)**2).mean()**(1/2),
                line_loss = ((sol.x[np.multiply(*anchors.shape):] - line_init)**2).mean()**(1/2),
                status = sol.status
            ))
            break

num_failures, retry_failure

Attempt:  1
Optimization terminated successfully.
         Current function value: 0.407294
         Iterations: 587
         Function evaluations: 11135
         Gradient evaluations: 655
Attempt:  1
         Current function value: 18.220192
         Iterations: 263
         Function evaluations: 6030
         Gradient evaluations: 354
Euclidean loss:  18.22019247637934
Attempt:  2


/Users/aldo.marini/miniforge3/envs/klipper/lib/python3.11/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)
/Users/aldo.marini/miniforge3/envs/klipper/lib/python3.11/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: 142.762326
         Iterations: 513
         Function evaluations: 12166
         Gradient evaluations: 715
Euclidean loss:  142.76232572014024
Attempt:  3
Optimization terminated successfully.
         Current function value: 1.644698
         Iterations: 307
         Function evaluations: 5746
         Gradient evaluations: 338
Euclidean loss:  1.644697906749259
Attempt:  4
Optimization terminated successfully.
         Current function value: 0.407799
         Iterations: 321
         Function evaluations: 6103
         Gradient evaluations: 359
Attempt:  1
Optimization terminated successfully.
         Current function value: 1.799339
         Iterations: 316
         Function evaluations: 5984
         Gradient evaluations: 352
Euclidean loss:  1.7993389074030426
Attempt:  2
Optimization terminated successfully.
         Current function value: 0.602852
         Iterations: 358
         Function evaluations: 6868
         Gradient evaluations: 404


/Users/aldo.marini/miniforge3/envs/klipper/lib/python3.11/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


Optimization terminated successfully.
         Current function value: 21.985503
         Iterations: 342
         Function evaluations: 6681
         Gradient evaluations: 393
Euclidean loss:  21.98550349021506
Attempt:  4
Optimization terminated successfully.
         Current function value: 0.393879
         Iterations: 140
         Function evaluations: 2584
         Gradient evaluations: 152
Attempt:  1
Optimization terminated successfully.
         Current function value: 0.589910
         Iterations: 341
         Function evaluations: 6460
         Gradient evaluations: 380
Attempt:  1
Optimization terminated successfully.
         Current function value: 0.573213
         Iterations: 204
         Function evaluations: 3774
         Gradient evaluations: 222
Attempt:  1
Optimization terminated successfully.
         Current function value: 0.606261
         Iterations: 163
         Function evaluations: 3060
         Gradient evaluations: 180
Attempt:  1
Optimization terminated 

/Users/aldo.marini/miniforge3/envs/klipper/lib/python3.11/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


KeyboardInterrupt: 

In [ ]:
'max anchor_loss', max([s['anchor_loss'] for s in sols[('BFGS', 'euclidean')]]), '\nmax line_loss', max([s['line_loss'] for s in sols[('BFGS', 'euclidean')]])

('max anchor_loss', 3482.4294390250707, '\nmax line_loss', 6024.365184332065)

In [ ]:
# num_points   failures    retry_failure
# 6 h 10-50    127         3
# 5 l 1 h 50   141         5
# 3 l 3 h      125         6
# 6 l 1 h 50   123         4
# 5 l 2 h 50 x 103         3
# 5 l 2 h 50   48, 55      0, 0
# 4 l 3 h 50 x 90          0
# 5 l 3 h 50   64          0
# 4 l 3 h 50   65, 48, 64  1, 1, 2
# 9            206         18
# 10 rand      187
# 10           213, 252    16, 24
# 9 l 1 h 50   55          0
# 10 h 10-11   73          1
# 10 h 10-15   11, 10      0
# 10 h 10-20   33          0
# 10 h 10-30   18          0
# 14 5 rand    52, 51      1
# 14           38, 43      0, 0
# 14 high 30   11          0


In [ ]:
stats = pd.DataFrame(sols[('BFGS', 'euclidean')])
len(stats[(stats.line_loss < 1) & (stats.anchor_loss < 1)])
# stats

NameError: name 'pd' is not defined

In [ ]:
import pandas as pd
res = {}
for name, values in sols.items():
    sol_list = values[0]    
    anchor_loss = np.mean([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_loss = np.mean([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    anchor_max = np.max([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_max = np.max([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    status = ''.join([str(sol.status) for sol in sol_list])
    res[name] = {'anchor_loss': anchor_loss, 'line_loss': line_loss, 'anchor_max': anchor_max, 'line_max': line_max, 'status': status}
pd.DataFrame(res)

,CG
,euclidean
anchor_loss,0.004993
anchor_max,0.004993
line_loss,0.008596
line_max,0.008596
status,0


In [ ]:
# quadratic euc
# MASSIVE gains from using two different z-levels for sampling euclidean
# 5 extra 10cm above euc non-random data points
import pandas as pd
res = {}
for name, sol_list in sols.items():
    anchor_loss = np.mean([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_loss = np.mean([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    anchor_max = np.max([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_max = np.max([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    status = ''.join([str(sol.status) for sol in sol_list])
    res[name] = {'anchor_loss': anchor_loss, 'line_loss': line_loss, 'anchor_max': anchor_max, 'line_max': line_max, 'status': status}
pd.DataFrame(res)

BFGS                                  CG              \
              euclidean       inter        both   euclidean       inter   
anchor_loss   11.428393    12.59487   11.429803    0.000151   11.780084   
line_loss     20.607131   12.359723   20.609428    0.000257   13.029805   
anchor_max    39.953429    13.83018   39.954191    0.000316   13.985962   
line_max      71.689287   13.693064   71.690534    0.000538   13.693064   
status       2222222222  0200000200  2222222222  2222222222  0002200000   

                         
                   both  
anchor_loss    0.001342  
line_loss       0.00228  
anchor_max     0.003117  
line_max       0.005301  
status       2222222222

In [ ]:
# MASSIVE gains from using two different z-levels for sampling euclidean
# 5 extra 10cm above euc non-random data points
import pandas as pd
res = {}
for name, sol_list in sols.items():
    anchor_loss = np.mean([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_loss = np.mean([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    anchor_max = np.max([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_max = np.max([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    status = ''.join([str(sol.status) for sol in sol_list])
    res[name] = {'anchor_loss': anchor_loss, 'line_loss': line_loss, 'anchor_max': anchor_max, 'line_max': line_max, 'status': status}
pd.DataFrame(res)

BFGS                                  CG              \
              euclidean       inter        both   euclidean       inter   
anchor_loss    0.000109   11.340306    0.000061    5.822015   12.106086   
line_loss      0.000185   12.071454    0.000103    9.605169   12.787283   
anchor_max     0.000496   13.864297    0.000112    7.745047   17.241418   
line_max       0.000839    13.76909    0.000192   13.076488   18.455082   
status       2222222222  2200022020  2222222222  2222222222  2200022020   

                         
                   both  
anchor_loss    6.003193  
line_loss      9.647818  
anchor_max     7.729335  
line_max      13.143444  
status       2222222222

In [ ]:
# 5 extra euc non-random data points
import pandas as pd
res = {}
for name, sol_list in sols.items():
    anchor_loss = np.mean([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_loss = np.mean([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    anchor_max = np.max([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_max = np.max([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    status = ''.join([str(sol.status) for sol in sol_list])
    res[name] = {'anchor_loss': anchor_loss, 'line_loss': line_loss, 'anchor_max': anchor_max, 'line_max': line_max, 'status': status}
pd.DataFrame(res)

BFGS                                  CG              \
              euclidean       inter        both   euclidean       inter   
anchor_loss    4.786648   11.109734    4.787982    6.228483   11.849605   
line_loss      8.725527   11.694716    8.727095    8.943039   12.619355   
anchor_max    23.992649   13.793859   24.025675    8.422153   21.285345   
line_max      43.685992   13.693064   43.719713   11.257996   18.011963   
status       2222222222  2022002000  2222222222  2222222222  2022002000   

                         
                   both  
anchor_loss    6.103912  
line_loss      8.878411  
anchor_max      8.41668  
line_max      11.247827  
status       2222222222

In [ ]:
# 5 extra euc data points
import pandas as pd
res = {}
for name, sol_list in sols.items():
    anchor_loss = np.mean([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_loss = np.mean([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    anchor_max = np.max([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_max = np.max([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    status = ''.join([str(sol.status) for sol in sol_list])
    res[name] = {'anchor_loss': anchor_loss, 'line_loss': line_loss, 'anchor_max': anchor_max, 'line_max': line_max, 'status': status}
pd.DataFrame(res)

BFGS                                  CG              \
              euclidean       inter        both   euclidean       inter   
anchor_loss    0.000021   11.888273    2.042821     6.20161   11.896541   
line_loss      0.000033   12.452719     4.06347    7.650643   12.389943   
anchor_max     0.000074   14.785774   20.427825     8.62187   14.785774   
line_max       0.000109   13.693064   40.634081    9.551657   13.693064   
status       2222222222  0200000002  2222222222  2222222222  0200000002   

                         
                   both  
anchor_loss    5.671906  
line_loss      7.711846  
anchor_max     8.669667  
line_max       9.470022  
status       2222222222

In [ ]:
# 10 extra euc data points
import pandas as pd
res = {}
for name, sol_list in sols.items():
    anchor_loss = np.mean([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_loss = np.mean([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    anchor_max = np.max([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_max = np.max([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    status = ''.join([str(sol.status) for sol in sol_list])
    res[name] = {'anchor_loss': anchor_loss, 'line_loss': line_loss, 'anchor_max': anchor_max, 'line_max': line_max, 'status': status}
pd.DataFrame(res)

BFGS                                  CG              \
              euclidean       inter        both   euclidean       inter   
anchor_loss    0.000025   11.792511      1.9475    5.197017   12.033498   
line_loss      0.000041   11.656645    3.944773     8.19232   11.428797   
anchor_max     0.000068    14.14206   19.472617    6.911751   15.165819   
line_max         0.0001   13.693064   39.443842   11.227904   13.693064   
status       2222222222  0002000222  2222222222  2222222222  0002000222   

                         
                   both  
anchor_loss    5.055432  
line_loss      8.123713  
anchor_max     6.974192  
line_max      11.379613  
status       2222222222

In [ ]:
# 100 extra euc data points
import pandas as pd
res = {}
for name, sol_list in sols.items():
    anchor_loss = np.mean([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_loss = np.mean([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    anchor_max = np.max([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_max = np.max([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    status = ''.join([str(sol.status) for sol in sol_list])
    res[name] = {'anchor_loss': anchor_loss, 'line_loss': line_loss, 'anchor_max': anchor_max, 'line_max': line_max, 'status': status}
pd.DataFrame(res)

Powell                               SLSQP              \
              euclidean       inter        both   euclidean       inter   
anchor_loss   11.787884   17.863724   12.157611    11.61201   10.881772   
line_loss      20.34181   13.066242   20.923199   20.918459   11.920202   
anchor_max    34.912919   42.225386   29.560119   43.331638   13.851175   
line_max      62.088163   15.619371   53.293789   77.957122   13.693064   
status       0000000100  0000000000  0000100001  0909999999  0000000000   

                        Nelder-Mead                                BFGS  \
                   both   euclidean       inter        both   euclidean   
anchor_loss   11.614585   15.054798   10.799138   14.187929    1.458247   
line_loss     20.922492   24.161673   10.146246   22.890797    2.874255   
anchor_max     43.35771   21.857299   13.195864   24.037809   14.581424   
line_max      78.004247   37.376439   13.693064   41.119562   28.740755   
status       9999099999  0000000000  0000000000  0000000000  2222222222   

                                             CG                          
                  inter        both   euclidean       inter        both  
anchor_loss   11.192262    1.457034    4.071498   11.374859    4.087257  
line_loss     11.952593    2.872626    6.959351   11.967763    6.985935  
anchor_max    14.105108   14.569517    5.654434   14.103146    5.698619  
line_max      13.693064    28.72486    9.667737   13.693064     9.74321  
status       0200222200  2222222222  2222222222  0200222200  2222222222

In [ ]:
import pandas as pd
res = {}
for name, sol_list in sols.items():
    anchor_loss = np.mean([((sol.x[:4*3].reshape(4,3) - anchors)**2).mean()**(1/2) for sol in sol_list])
    line_loss = np.mean([((sol.x[4*3:] - line_init)**2).mean()**(1/2) for sol in sol_list])
    status = ''.join([str(sol.status) for sol in sol_list])
    res[name] = {'anchor_loss': anchor_loss, 'line_loss': line_loss, 'status': status}
pd.DataFrame(res)

Powell                               SLSQP              \
              euclidean       inter        both   euclidean       inter   
anchor_loss   18.654866    16.48032   20.630563   29.746909   20.726336   
line_loss     13.675645   14.417241   13.689917    52.15793   14.432158   
status       0000000000  0000000000  0000000000  9999999999  0000000000   

                        Nelder-Mead                                BFGS  \
                   both   euclidean       inter        both   euclidean   
anchor_loss   28.579247   11.074836   13.014282   11.052235    8.934511   
line_loss     49.086452   11.693474     9.97408   11.664373   13.124035   
status       9990999999  0000000000  0000000000  0000000000  2222222222   

                                             CG                          
                  inter        both   euclidean       inter        both  
anchor_loss   11.984731   11.802847   10.409172    11.94372   10.685206  
line_loss     11.522966   19.327263    9.659414   11.403273    9.539519  
status       2220202200  2222222222  2222222222  2220202200  2222222222

In [ ]:
print(sol1.x[:4*3].reshape(4,3) - anchors, '\n', sol1.x[:4*3].reshape(4,3), '\n',anchors)
print('----------------------------------')
print(sol1.x[4*3:] - line_init, '\n', sol1.x[4*3:], '\n', line_init)

NameError: name 'sol1' is not defined

In [ ]:
inter_anchor_and_euclidean(sol0.x, np.concatenate([measured_line_lengths, hat_line_lengths], axis=1),
                           num_points, grid_points)

inter 9.36899292870272
euc 3014.5802290045817


3023.9492219332847

In [ ]:
inter_anchor_and_euclidean(sol1.x, np.concatenate([measured_line_lengths, hat_line_lengths], axis=1),
                           num_points, grid_points)

inter 6.189493883656275
euc 8977.562060882614


8983.75155476627